In [5]:
import json
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.discriminant_analysis import StandardScaler
from sklearn.svm import OneClassSVM
import matplotlib.pyplot as plt
from pprint import pprint 
from IPython.display import display
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import IsolationForest



In [17]:

eve_path = "../server/demoData/eve.json"

# Load Suricata JSON log
with open(eve_path, 'r') as f:
    data = [json.loads(line) for line in f]

# Create DataFrame
df = pd.json_normalize(data, errors='raise', record_prefix='event_')
display(df)



,timestamp,flow_id,in_iface,event_type,src_ip,src_port,dest_ip,dest_port,proto,dns.type,...,fileinfo.end,rfb.server_protocol_version.major,rfb.server_protocol_version.minor,rfb.client_protocol_version.major,rfb.client_protocol_version.minor,rfb.authentication.security_type,dns.grouped.PTR,krb5.failed_request,krb5.error_code,smb.ntlmssp.version
0,2024-11-23T11:10:47.275437-0800,1.051748e+15,eth1,dns,10.129.180.205,58911.0,192.168.220.254,53.0,UDP,query,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-11-23T11:10:47.275418-0800,1.531850e+15,eth0,dns,10.129.180.205,58911.0,10.100.105.254,53.0,UDP,query,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2024-11-23T11:10:47.276172-0800,1.051748e+15,eth1,dns,10.129.180.205,58911.0,192.168.220.254,53.0,UDP,answer,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2024-11-23T11:10:47.276184-0800,1.531850e+15,eth0,dns,10.129.180.205,58911.0,10.100.105.254,53.0,UDP,answer,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2024-11-23T11:10:47.297166-0800,2.481553e+14,eth0,dns,10.100.105.254,60624.0,8.8.8.8,53.0,UDP,query,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53178,2024-11-23T15:51:20.917132-0800,1.562165e+15,eth1,fileinfo,23.61.213.219,80.0,192.168.220.254,59521.0,TCP,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53179,2024-11-23T15:51:20.913632-0800,1.920484e+15,eth0,fileinfo,10.100.105.254,59521.0,23.61.213.219,80.0,TCP,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53180,2024-11-23T15:51:22.391960-0800,2.078777e+15,eth1,flow,192.168.220.111,50908.0,224.0.0.252,5355.0,UDP,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53181,2024-11-23T15:51:20.917125-0800,1.920484e+15,eth0,fileinfo,23.61.213.219,80.0,10.100.105.254,59521.0,TCP,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Ensure all columns contain only scalar values
df_clean = df.applymap(lambda x: x if isinstance(x, (int, float, str)) else None)

# Select columns with object type
categorical_cols = df_clean.select_dtypes(include=['object']).columns

# Frequency encoding for categorical columns
for col in categorical_cols:
	freq_encoding = df_clean[col].value_counts().to_dict()
	df_clean[col] = df_clean[col].map(freq_encoding)

# Drop any remaining non-scalar columns
df_clean = df_clean.dropna(axis=1, how='any')

# The resulting DataFrame is df_clean
df_encoded = df_clean
    

C:\Users\khawa\AppData\Local\Temp\ipykernel_23260\2907467303.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_clean = df.applymap(lambda x: x if isinstance(x, (int, float, str)) else None)


,10095,1554,1987,21084,239,316,370,457,6508,699,83,86,87,9617
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53178,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
53179,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
53180,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
53181,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


C:\Users\khawa\AppData\Local\Temp\ipykernel_23260\2973211686.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_clean = df_encoded.applymap(lambda x: x if isinstance(x, (int, float, str)) else None).apply(pd.to_numeric, errors="coerce")


KeyboardInterrupt: 

In [ ]:
# Ensure the dataframe does not contain any non-scalar values

model = IsolationForest(contamination=0.3)  
model.fit(df_clean)

df['anomaly'] = model.predict(df_clean)

anomalies = df[df['anomaly'] == -1]
display(anomalies)

# Save anomalies to JSON file
anomalies.to_json('anomalies.json', orient="records", lines=True)

,timestamp,flow_id,in_iface,event_type,src_ip,src_port,dest_ip,dest_port,proto,dns.type,...,rfb.server_protocol_version.major,rfb.server_protocol_version.minor,rfb.client_protocol_version.major,rfb.client_protocol_version.minor,rfb.authentication.security_type,dns.grouped.PTR,krb5.failed_request,krb5.error_code,smb.ntlmssp.version,anomaly
8,2024-11-23T11:10:47.319773-0800,1.131048e+15,eth0,ssh,10.129.180.205,60406.0,10.100.105.44,22.0,TCP,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
9,2024-11-23T11:10:47.319760-0800,1.846667e+15,eth1,ssh,10.129.180.205,60406.0,192.168.220.44,22.0,TCP,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
11,2024-11-23T11:10:47.321748-0800,7.755623e+14,eth0,anomaly,10.100.105.231,25.0,10.129.180.205,59220.0,TCP,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
12,2024-11-23T11:10:47.321941-0800,6.475271e+14,eth0,fileinfo,10.100.105.42,80.0,10.129.180.205,52206.0,TCP,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
13,2024-11-23T11:10:47.329363-0800,8.117817e+14,eth0,ssh,10.129.180.205,42258.0,10.100.105.221,22.0,TCP,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53177,2024-11-23T15:51:20.913632-0800,1.920484e+15,eth0,http,10.100.105.254,59521.0,23.61.213.219,80.0,TCP,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
53178,2024-11-23T15:51:20.917132-0800,1.562165e+15,eth1,fileinfo,23.61.213.219,80.0,192.168.220.254,59521.0,TCP,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
53179,2024-11-23T15:51:20.913632-0800,1.920484e+15,eth0,fileinfo,10.100.105.254,59521.0,23.61.213.219,80.0,TCP,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
53181,2024-11-23T15:51:20.917125-0800,1.920484e+15,eth0,fileinfo,23.61.213.219,80.0,10.100.105.254,59521.0,TCP,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
